<a href="https://colab.research.google.com/github/Martinaeht/Irony_Detection_roberta_bertweet/blob/master/Irony_Detection_MT_RoBERTa_Optuna_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IRONY DETECTION

**Dataset:** https://huggingface.co/datasets/cardiffnlp/tweet_eval

**Model:** https://huggingface.co/FacebookAI/roberta-base




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade
!pip install optuna
!pip install emoji==0.6.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 5.0 MB/s eta 0:00:00
  Att

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, RobertaModel, RobertaForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, set_seed
from transformers import AutoModelForMaskedLM
from pprint import pprint
import numpy as np
import evaluate
import torch
from torch.utils.tensorboard import SummaryWriter
import os
import optuna
import json
from sklearn.metrics import precision_score, recall_score, f1_score


set_seed(333) ## global seed

irony_dataset = load_dataset("cardiffnlp/tweet_eval", "irony")

train_dataset = irony_dataset['train']
validation_dataset = irony_dataset['validation']
test_dataset = irony_dataset['test']

train_dataset_print = train_dataset[:10]

for label, text in zip(train_dataset_print['label'], train_dataset_print['text']):
    print(label, text)

small_irony_dataset = DatasetDict(
    train = train_dataset.shuffle(seed=333).select(range(640)), # local seed
    val = validation_dataset.shuffle(seed=333).select(range(160)),
    test = test_dataset.shuffle(seed=333).select(range(160)))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/183k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/54.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2862 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/784 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/955 [00:00<?, ? examples/s]

1 seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life
0 look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵
1 Now I remember why I buy books online @user #servicewithasmile
1 @user @user So is he banded from wearing the clothes?  #Karma
1 Just found out there are Etch A Sketch apps.  #oldschool #notoldschool
1 Hey what do you know, one of the witnesses supporting Darren Wilson's story lied! And is racist! Mind blown!
0 @user on stage at #flzjingleball at the @user in #Tampa #iheartradio
1 You know it's going to be a great day when you're Garmin resets itself and you spill some cinnamon down yourself  #slowclap
1 Halfway thorough my workday ... Woooo
1 Would like to thank my nephew for giving me his horrible cold & sore throat etc.. Much appreciated!


# RoBERTa

https://huggingface.co/docs/transformers/main/en/model_doc/roberta

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
#tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

max_length = 128

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",  # Pad to max_length
        truncation=True,       # Truncate if longer
        max_length=max_length  # Set max length
    )

#def tokenize_function(examples):
#    return tokenizer(examples["text"], padding=True, truncation=True) # added truncation = True

small_tokenized_dataset = small_irony_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

#model = RobertaModel.from_pretrained("FacebookAI/roberta-base", num_labels=2)
#model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion"
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-base", num_labels=2)

accuracy = evaluate.load("accuracy")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# RoBERTa Optuna

In [ ]:
def compute_metrics(eval_pred): # added
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = (predictions == labels).mean()
  return {"accuracy": accuracy}

In [ ]:
# Optuna objective function

def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    num_train_epochs = trial.suggest_int('num_train_epochs', 3, 6)
    weight_decay = trial.suggest_uniform('weight_decay', 0.0, 0.1)

    arguments = TrainingArguments(
        output_dir="sample_cl_trainer_rob_640",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_steps=8,
        num_train_epochs=num_train_epochs,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        report_to='none',
        seed=333,
        #lr_scheduler_type='linear',  # Added scheduler
        warmup_steps=500  # Added warmup steps to avoid early overfitting
    )

    trainer = Trainer(
        model=model,
        args=arguments,
        train_dataset=small_tokenized_dataset['train'],
        eval_dataset=small_tokenized_dataset['val'],
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()

    eval_results = trainer.evaluate()

    return eval_results["eval_accuracy"]


In [ ]:
# Optuna study

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print(f"Best trial: {study.best_trial.params}")


[I 2025-01-27 16:45:21,399] A new study created in memory with name: no-name-0eb02f85-4cdd-48a9-9382-a0ed66de7be2
<ipython-input-6-de604ad1c713>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
<ipython-input-6-de604ad1c713>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.0, 0.1)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.692100,0.687845,0.568750
2,0.695400,0.687980,0.568750
3,0.692300,0.687597,0.568750
4,0.689300,0.685811,0.568750
5,0.688800,0.682793,0.568750
6,0.682400,0.677475,0.568750


[I 2025-01-27 16:47:49,796] Trial 0 finished with value: 0.56875 and parameters: {'learning_rate': 1.6961019976076524e-05, 'batch_size': 32, 'num_train_epochs': 6, 'weight_decay': 0.04945195296111835}. Best is trial 0 with value: 0.56875.
<ipython-input-6-de604ad1c713>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
<ipython-input-6-de604ad1c713>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.0, 0.1)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.678800,0.677071,0.568750
2,0.648500,0.675527,0.637500
3,0.643400,0.674629,0.581250
4,0.619300,0.683709,0.568750
5,0.538600,0.671444,0.568750


[I 2025-01-27 16:50:20,779] Trial 1 finished with value: 0.56875 and parameters: {'learning_rate': 1.0934528330738551e-05, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.009760721897862491}. Best is trial 0 with value: 0.56875.
<ipython-input-6-de604ad1c713>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
<ipython-input-6-de604ad1c713>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.0, 0.1)


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.133300,0.715979,0.687500
2,0.230000,0.755441,0.662500
3,0.213300,0.882636,0.637500
4,0.122300,0.890934,0.687500


[I 2025-01-27 17:05:25,660] Trial 8 finished with value: 0.6875 and parameters: {'learning_rate': 1.2147380242556548e-05, 'batch_size': 16, 'num_train_epochs': 4, 'weight_decay': 0.011206027591334223}. Best is trial 4 with value: 0.725.
<ipython-input-6-de604ad1c713>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
<ipython-input-6-de604ad1c713>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.0, 0.1)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.112000,0.804557,0.662500
2,0.214600,0.805565,0.693750
3,0.146200,1.092436,0.637500
4,0.100700,1.376036,0.656250
5,0.084600,1.436028,0.718750


[I 2025-01-27 17:07:53,588] Trial 9 finished with value: 0.6625 and parameters: {'learning_rate': 2.9372079373243932e-05, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.054532234830701735}. Best is trial 4 with value: 0.725.


Best trial: {'learning_rate': 2.9083932119951922e-05, 'batch_size': 8, 'num_train_epochs': 6, 'weight_decay': 0.06048928933132049}


In [ ]:
save_directory = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_optuna_640"
if not os.path.exists(save_directory):
  os.makedirs(save_directory)

best_params_path = f"{save_directory}/best_params.json"
with open(best_params_path, 'w') as f:
    json.dump(study.best_trial.params, f)

print(f"Best trial parameters saved to {best_params_path}")

tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

Best trial parameters saved to /content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_optuna_640/best_params.json


In [ ]:

with open(best_params_path, 'r') as f:
    best_params = json.load(f)

print("Best trial parameters loaded: ", best_params)

optuna_model = RobertaForSequenceClassification.from_pretrained(save_directory, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

#best_params = study.best_trial.params

# Final Training with optimal hyperparameters
arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_final_model_trainer_640",
    per_device_train_batch_size=best_params['batch_size'],
    per_device_eval_batch_size=best_params['batch_size'],
    logging_steps=8,
    num_train_epochs=best_params['num_train_epochs'],
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_params['learning_rate'],
    weight_decay=best_params['weight_decay'],
    load_best_model_at_end=True,
    report_to='none',
    seed=333
)

trainer = Trainer(
    model=optuna_model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


Best trial parameters loaded:  {'learning_rate': 2.9083932119951922e-05, 'batch_size': 8, 'num_train_epochs': 6, 'weight_decay': 0.06048928933132049}


Epoch,Training Loss,Validation Loss,Accuracy
1,0.407600,1.021682,0.618750
2,0.320200,1.858651,0.600000
3,0.347700,1.643142,0.668750
4,0.005900,2.069323,0.687500
5,0.000100,2.115823,0.681250
6,0.000100,2.314243,0.675000


TrainOutput(global_step=480, training_loss=0.15579671499738953, metrics={'train_runtime': 180.0194, 'train_samples_per_second': 21.331, 'train_steps_per_second': 2.666, 'total_flos': 252586613145600.0, 'train_loss': 0.15579671499738953, 'epoch': 6.0})

# roBERTa - testing the finetuned model

In [ ]:
##best epoch

output_dir_test = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_testing_finetuned_model_640_bestep_cp320"
if not os.path.exists(output_dir_test):
  os.makedirs(output_dir_test)

#adapted checkpoint
fine_tuned_model = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_final_model_trainer_640/checkpoint-320", num_labels=2)


for example in small_tokenized_dataset['test']:
  text = example['text']
  model_inputs = tokenizer(text, return_tensors="pt")
  prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
  predicted_label = ["non-irony", "irony"][prediction]
  print(f"Text: {text}\nPredicted Label: {predicted_label}\n")


arguments_test_set = TrainingArguments(
    output_dir="results_test_set_opt640",
    per_device_eval_batch_size=16,
    report_to='none',
    seed=333
)

trainer_test_set = Trainer(
    model=fine_tuned_model,
    args=arguments_test_set,
    eval_dataset=small_tokenized_dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results = trainer_test_set.evaluate(eval_dataset=small_tokenized_dataset['test'])
print(f"Test Accuracy: {test_results['eval_accuracy']}")

#output: Test Accuracy: 0.68125
#compared to training accuracy: 0.687500 in best epoch (4/cp320)



NameError: name 'os' is not defined

In [ ]:
# F1 score

def compute_f1_score(model, tokenizer, dataset):
    all_labels = []
    all_predictions = []

    for example in dataset:
        text = example['text']
        label = example['label']
        model_inputs = tokenizer(text, return_tensors="pt").to(device) # Move model_inputs to the device
        logits = model(**model_inputs).logits
        prediction = torch.argmax(logits, dim=-1).item()  # Get single prediction

        all_labels.append(label)
        all_predictions.append(prediction)

    precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_predictions, average='weighted', zero_division=0)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

print(compute_f1_score(fine_tuned_model, tokenizer, small_tokenized_dataset['test']))


{'precision': 0.6791577518104015, 'recall': 0.68125, 'f1': 0.6799625678266901}


worst epoch

In [ ]:
# worst epoch

output_dir_test_worstep = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_testing_finetuned_model_640_worstep_cp160"
if not os.path.exists(output_dir_test_worstep):
  os.makedirs(output_dir_test_worstep)

fine_tuned_model_worstep = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_final_model_trainer_640/checkpoint-160", num_labels=2)


for example in small_tokenized_dataset['test']:
  text = example['text']
  model_inputs = tokenizer(text, return_tensors="pt")
  prediction = torch.argmax(fine_tuned_model_worstep(**model_inputs).logits)
  predicted_label = ["non-irony", "irony"][prediction]
  print(f"Text: {text}\nPredicted Label: {predicted_label}\n")


arguments_test_set = TrainingArguments(
    output_dir="results_test_set_640_worstep",
    per_device_eval_batch_size=16,
    report_to='none',
    seed=333
)

trainer_test_set = Trainer(
    model=fine_tuned_model_worstep,
    args=arguments_test_set,
    eval_dataset=small_tokenized_dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results_worstep = trainer_test_set.evaluate(eval_dataset=small_tokenized_dataset['test'])
print(f"Test Accuracy: {test_results_worstep['eval_accuracy']}")


#output: Test Accuracy: 0.65
#compared to worst training accuracy: 0.600000


Text: #AdvancedWarfare is so much better than any FPS released in 2014 XD
Predicted Label: irony

Text: @user @user @user |*Everything is crossed*|#ChristmasGift |#BassFace
Predicted Label: non-irony

Text: Laborers in the San Joaquin Valley, one of the most productive farming regions in the world, suffer from food insecurity. #irony #poverty
Predicted Label: irony

Text: Pips who drop out of school n have Kirya as 'eir role model tht ey wana sing too..guy spix Portuguese,French,english...#Not a school dropout
Predicted Label: non-irony

Text: I really want to see Into The Woods!
Predicted Label: irony

Text: #Irony: al jazeera is pro Anti - #GamerGate because feminism, or something:
Predicted Label: irony

Text: Sleep. SLEEP YOU STUPID IDIOT I WANNA LEAVE MY ROOM
Predicted Label: irony

Text: Some days are just better than others..
Predicted Label: irony

Text: Christmas alone😊 how nice #not
Predicted Label: irony

Text: Read at the news '#Obama: #The #American #way #of #life #is #not

Test Accuracy: 0.65


# roBERTa Visualization

https://projector.tensorflow.org/

In [ ]:
# Tensors on different devices --> solution:

# Determine the device (CUDA or CPU)
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the correct device
#fine_tuned_model.to(device)

# Tokenize the input text and move it to the same device
model_inputs = tokenizer(small_tokenized_dataset['test']['text'], padding=True, truncation=True, return_tensors='pt')
model_inputs = {key: value.to(device) for key, value in model_inputs.items()}

outputs = fine_tuned_model(**model_inputs, output_hidden_states=True)


In [ ]:
# best epoch

path = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_results_vis_640_bestep_cp320"
if not os.path.exists(path):
  os.mkdir(path)

layer=0

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path+'/layer_' + str(layer)):
    os.mkdir(path+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
        tensor = outputs['hidden_states'][layer][example][0] # [sp_token_position]
        tensors.append(tensor)
        label = [small_tokenized_dataset['test']['text'][example],str(small_tokenized_dataset['test']['label'][example])]
        labels.append(label)
        example +=1

  writer=SummaryWriter(path+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['text','label'])

  layer+=1


worst epoch

In [ ]:

model_inputs = tokenizer(small_tokenized_dataset['test']['text'], padding=True, truncation=True, return_tensors='pt')
model_inputs = {key: value.to(device) for key, value in model_inputs.items()}

outputs = fine_tuned_model_worstep(**model_inputs, output_hidden_states=True)


NameError: name 'tokenizer' is not defined

In [ ]:
# worst epoch

path_worstep = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_results_vis_640_worstep_cp160"
if not os.path.exists(path_worstep):
  os.mkdir(path_worstep)

layer=0

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path_worstep+'/layer_' + str(layer)):
    os.mkdir(path_worstep+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
        tensor = outputs['hidden_states'][layer][example][0] # [sp_token_position]
        tensors.append(tensor)
        label = [small_tokenized_dataset['test']['text'][example],str(small_tokenized_dataset['test']['label'][example])]
        labels.append(label)
        example +=1

  writer=SummaryWriter(path_worstep+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['text','label'])

  layer+=1
